<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_KoSpacing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 띄어쓰기 검사 모델 구현하기

## 필요한 패키지 다운로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
pip install soynlp

In [ ]:
pip install customized_konlpy

In [37]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-stq1md9a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-stq1md9a
     |████████████████████████████████| 132 kB 7.3 MB/s 
     |████████████████████████████████| 344 kB 62.6 MB/s 
     |████████████████████████████████| 47.3 MB 58 kB/s 
     |████████████████████████████████| 4.5 MB 40.2 MB/s 
     |████████████████████████████████| 1.2 MB 32.9 MB/s 
     |████████████████████████████████| 3.8 MB 38.2 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 6.5 MB 50.6 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 8.6 MB 15.1 MB/s 
     |████████████████████████████████| 127 kB 73.3 MB/s 
  Created wheel for kobert: filename=kobe

In [2]:
import os

In [3]:

from hanspell import spell_checker

import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *

## 데이터 불러오기

In [4]:
root = '/content/drive/MyDrive/news_class9x1400/'
categories = os.listdir(root)

dataset = []

for cat in categories:
  files = os.listdir(root + cat)
  for i, f in enumerate(files):
    fname = root + cat + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()

print(len(dataset))

12600


In [6]:
import random

random.shuffle(dataset)
mini_dataset = dataset[:1000]

In [7]:
mini_dataset[241]

['찜통더위 물렀거라!\n낮잠이 보약\n산에서 낮잠 자는 사람들이 왜 이렇게 많아?\n그러게 말이야, 낮잠은 방안이나 정자에서 자야 되는 거 아냐?\n낮잠 자는 장소가 따로 정해져 있나?\n졸리면 자는 거지.\n일행들이 한 마디씩 한다.\n서울 관악구 신림동 관악산에서 능선으로 이어진 삼성산 등산길에는 낮잠 자는 사람들이 많았다.\n전에도 이 산을 많이 올랐지만 이런 풍경은 처음이었다.\n부부끼리, 친구끼리, 또는 홀로 나무그늘에 돗자리를 펴고 들어 누워 낮잠 자는 사람들은 자세도 다양했다.\n64회 광복절이자 주말인 15일 친구들 몇 사람과 함께 관악산 등산에 나섰다.\n일행들은 서울대학교 정문 쪽 등산로 입구 광장 시계탑 밑에서 만났다.\n친구들은 이번에도 관악산이 아닌 삼성산으로 올라가잔다.\n이유는 날씨가 너무 무덥다는 것이었다.\n주차장에서 곧장 아파트 옆 골목으로 빠져 등산로로 나섰다.\n그런데 무더위가 정말 장난이 아니었다.\n서울지방의 기온이 섭씨 33도가 넘을 것이라는 일기예보가 실감나는 날씨였다.\n땡볕은 그렇다 해도 나무그늘이 드리워진 산길도 푹푹 찌는 열기가 온몸을 휘감고 있었다.\n히야! 이거 더워 죽겠네, 이쪽으로 오르길 잘했지, 이런 날씨에 저쪽 관악산이었으면 어쩔 뻔 했어?\n잠깐 쉬어가지?\n경사가 완만한 길인데도 얼마 올라가지 못하고 땀으로 뒤범벅이 된 얼굴로 모두들 길가에 주저앉는다.\n잠시 쉬었다가 다시 오르기 시작했다.\n그런데 약간 가파른 길을 올라도 숨이 헉헉 턱에 찬다.\n너무 무더운 날씨 때문이었다.\n그렇게 허위허위 청소년야영장이 있는 능선길에 올라섰다.\n그런데 능선길에 올라서자 사정없이 내려 쏟아지는 햇볕이 그야말로 불덩이다.\n햇볕을 피해 나무그늘 밑으로 숨어들자 곳곳에 낮잠 자는 사람들이 눈에 띄기 시작했다.\n부부로 보이는 어떤 커플은 나무그늘이 아니라 땡볕에서 깊이 잠든 모습이다.\n그늘 밑에서 잠들었다가 그늘이 옮겨진 것도 모른 채 계속 자고 있었기 때문일 것이다.\n조금 더 걸어가자 이번엔 역시 부부

## 데이터 전처리

In [8]:
!pip install kss

In [9]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': ''

})
test = '"안녕"하세★요? 좋은 아▲침입┌니├다. 오│늘도 힘내└세요→!'
test = test.translate(table)

test

'안녕하세요? 좋은 아침입니다. 오늘도 힘내세요!'

In [ ]:
import kss
import random

process_dataset = []

for sentences in mini_dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split("\n")

  for sentence in sentences:
    process_dataset.extend(kss.split_sentences(sentence))


len(process_dataset)

In [21]:
# 4h 36m 

len(process_dataset)

22003

In [22]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(process_dataset, test_size = 0.2, random_state  = 42)
print(len(train), len(test))

17602 4401


In [ ]:
def save_datafile(file_name, dataset):
  with open(
      os.path.join(root, file_name), mode = 'w', encoding='utf-8'
  ) as f:
    for data in dataset:
      f.write(data)

save_datafile('train.txt', train)
save_datafile('test.txt', test)

print('train, test dataset 저장 완료!')

## 띄어쓰기를 위한 클래스

In [33]:
from typing import List

def load_slot_labels() -> List[str]:
    """tag label 종류 리턴"""
    return ["UNK", "PAD", "O", "B", "I", "E", "S"]

In [10]:
from typing import Callable, List, Tuple
from torch.utils.data import Dataset

class CorpusDataset(Dataset):
  def __init__(self, data_path: str, transform: Callable[[List, List], Tuple]):
    self.sentences = []
    self.slot_labels = ['UNK', 'PAD', 'B', 'I']
    self._load_data(data_path)
    self.transform = transform

  def _load_data(self, data_path: str):
    with open(data_path, mode = 'r', encoding = 'utf-8') as f:
      lines = f.readlines()
      self.sentences = [line.split() for line in lines]

  def _get_tags(self, sentences: List[str]) -> List[str]:
    tags = []
    for word in sentence:
      for i in range(len(word)):
        if i == 0:
            tags.append("B")
        else:
            tags.append("I")
    return tags

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self, idx):
    sentence = "".join(self.sentences[idx])
    sentence = [s for s in sentence]
    tags = self._get_tags(self.sentences[idx])
    tags = [self.slot_labels.index(t) for t in tags]

    (
        input_ids,
        attention_mask,
        token_type_ids,
        slot_label_ids, 
    ) = self.transform(sentence, tags)

    return input_ids, attention_mask, token_type_ids, slot_label_ids

In [13]:
pip install transformers

In [16]:
pip install tokenizers

### Tokenization_kobert


In [21]:
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBERT model """


import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "tokenizer_78b3253a26.model",
    "vocab_txt": "vocab.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model",
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False},
}

SPIECE_UNDERLINE = "▁"


class KoBertTokenizer(PreTrainedTokenizer):
    """
    SentencePiece based tokenizer. Peculiarities:
        - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        vocab_txt,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        **kwargs,
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, "r", encoding="utf-8") as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize("NFKD", outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text):
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(
                map(
                    lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0,
                    token_ids_0,
                )
            )

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """Save the sentencepiece vocabulary (copy original file) and special tokens file
        to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [22]:
import torch

class Preprocessor:
    def __init__(self, max_len: int):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(
        self, sentence: List[str], tags: List[str]
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.

        Args:
            sentence: 문장
            tags: 띄어쓰기 tagging

        Returns:
            feature를 리턴한다.
            input_ids, attention_mask, token_type_ids, slot_labels
        """

        input_tokens = []
        slot_label_ids = []
					
        # tokenize
        for word, tag in zip(sentence, tags):
            tokens = self.tokenizer.tokenize(word)

            if len(tokens) == 0:
                tokens = self.tokenizer.unk_token

            input_tokens.extend(tokens)

            for i in range(len(tokens)):
                if i == 0:
                    slot_label_ids.extend([tag])
                else:
                    slot_label_ids.extend([self.pad_token_id])

        # max_len보다 길이가 길면 뒤에 자르기
        if len(input_tokens) > self.max_len - 2:
            input_tokens = input_tokens[: self.max_len - 2]
            slot_label_ids = slot_label_ids[: self.max_len - 2]

        # cls, sep 추가
        input_tokens = (
            [self.tokenizer.cls_token] + input_tokens + [self.tokenizer.sep_token]
        )
        slot_label_ids = [self.pad_token_id] + slot_label_ids + [self.pad_token_id]

        # token을 id로 변환
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        # padding
        pad_len = self.max_len - len(input_tokens)
        input_ids = input_ids + ([self.tokenizer.pad_token_id] * pad_len)
        slot_label_ids = slot_label_ids + ([self.pad_token_id] * pad_len)
        attention_mask = attention_mask + ([0] * pad_len)
        token_type_ids = token_type_ids + ([0] * pad_len)

        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        slot_label_ids = torch.tensor(slot_label_ids, dtype=torch.long)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

In [ ]:
preprocessor = Preprocessor(config.max_len)
dataset = CorpusDataset(data_path, preprocessor.get_input_features)

## 모델 생성

In [ ]:
pip install pytorch_lightning

In [ ]:
pip install seqeval

In [30]:
pip install utils

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import BertConfig, BertModel, AdamW
from torch.utils.data import DataLoader
from seqeval.metrics import f1_score


class SpacingBertModel(pl.LightningModule):
    def __init__(
        self,
        config,
        dataset: CorpusDataset,
    ):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.slot_labels_type = ["UNK", "PAD", "B", "I"]
        self.pad_token_id = 0

        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=len(self.slot_labels_type)
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        x = outputs[0]
        x = self.dropout(x)
        x = self.linear(x)

        return x

    def training_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_labels)
        tensorboard_logs = {"train_loss": loss}

        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids)
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        val_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)

        return {"val_loss": loss, "val_f1": val_f1}

    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()

        tensorboard_log = {
            "val_loss": val_loss,
            "val_f1": val_f1,
        }

        return {"val_loss": val_loss, "progress_bar": tensorboard_log}

    def test_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        test_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)

        test_step_outputs = {
            "test_f1": test_f1,
        }

        return test_step_outputs

    def test_epoch_end(self, outputs):
        test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()

        test_step_outputs = {
            "test_f1": test_f1,
        }

        return test_step_outputs

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.train_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def _calculate_loss(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        loss = F.cross_entropy(active_logits, active_labels)

        return loss

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

## 모델 학습

In [35]:
train_data_path = '/content/drive/MyDrive/train_test/train.txt'
test_data_path = '/content/drive/MyDrive/train_test/test.txt'

In [38]:
pip install omegaconf 

     |████████████████████████████████| 74 kB 2.6 MB/s 
     |████████████████████████████████| 112 kB 19.1 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=3bb4a82a12a04379d611eafb7785b9d50428b4ca255c32b1221d3f4e5d7a7938
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [51]:
from omegaconf import OmegaConf

config = OmegaConf.load('/content/drive/MyDrive/train_config.yaml')

In [52]:
dataset = {}
dataset["train"] = CorpusDataset(
    train_data_path, Preprocessor.get_input_features
)
dataset["test"] = CorpusDataset(
    test_data_path, Preprocessor.get_input_features
)


bert_finetuner = SpacingBertModel(config, dataset)

In [53]:
from pytorch_lightning.loggers import TensorBoardLogger
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=1, name=config.task
)

checkpoint_callback = ModelCheckpoint(
    filepath="checkpoints/"+ config.task + "/{epoch}_{val_loss:35f}",
    verbose=True,
    monitor="val_loss",
    mode="min",
    save_top_k=3,
    prefix="",
)

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=3,
    verbose=False,
    mode="min",
)

In [54]:
trainer = pl.Trainer(
    gpus=config.gpus,
    distributed_backend=config.distributed_backend,
    checkpoint_callback=checkpoint_callback,
    early_stop_callback=early_stop_callback,
    logger=logger,
)

trainer.fit(bert_finetuner)
trainer.test()

TypeError: ignored